# Basic testing of the library

In [1]:
from SynGenLoss.Model1 import AabjoraModel 
import numpy as np

In [2]:
G1 = AabjoraModel() 

In [3]:
P = 0.9 
Q = np.tan(np.arccos(P))*P
Ia, If, delta = G1.calc_currents(0.9, Q, 1.0)
print(f"Ia = {(Ia*5406):.2f} A, If = {(If*525.15):.2f} A, delta = {delta:.2f} rad")

Ia = 5406.00 A, If = 1064.93 A, delta = 0.44 rad


In [4]:
n_real = 0.98834 
P_loss_real = 1094.02 #kW
n, P_st, P_rot, P_con = G1.calc_losses_pu(P, Q, 1.0)
P_loss = P_st + P_rot + P_con 
print(f"Deviations: n = {(n*100-n_real*100):.2f} %, P_loss = {(P_loss*103000-P_loss_real):.2f}")

Deviations: n = 0.00 %, P_loss = -1.02


In [5]:
P_loss*103000

1092.9967400634462

In [6]:
173.65 + 15.88 + 2.13 

191.66